In [ ]:
!pip install transformers datasets scikit-learn pandas torch

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving requirements.csv to requirements.csv


In [ ]:
import pandas as pd

df = pd.read_csv("requirements.csv")
print(df.head())

# نحول التصنيفات إلى أرقام
label_map = {"Functional": 0, "Non-Functional": 1}
df["label"] = df["label"].map(label_map)
df.head()

                                     requirement           label
0  The owner can change the access code anytime.      Functional
1               The door should respond quickly.  Non-Functional
2           Only the tenant can unlock the door.      Functional
3    The system must support logging activities.      Functional
4                      The app should look good.  Non-Functional


,requirement,label
0,The owner can change the access code anytime.,0
1,The door should respond quickly.,1
2,Only the tenant can unlock the door.,0
3,The system must support logging activities.,0
4,The app should look good.,1


In [ ]:
import pandas as pd

# قراءة الملف
df = pd.read_csv("requirements.csv", encoding="utf-8")

print(f"📄 عدد المتطلبات: {len(df)}")
display(df.head())

# تحويل التصنيفات النصية إلى أرقام
label_map = {"Functional": 0, "Non-Functional": 1}
df["label"] = df["label"].map(label_map)

📄 عدد المتطلبات: 20


,requirement,label
0,The owner can change the access code anytime.,Functional
1,The door should respond quickly.,Non-Functional
2,Only the tenant can unlock the door.,Functional
3,The system must support logging activities.,Functional
4,The app should look good.,Non-Functional


In [ ]:
df = pd.read_csv("requirements.csv", encoding="utf-8")
df = df.dropna(subset=["requirement"])  # حذف الصفوف الفارغة
print(f"📄 عدد المتطلبات بعد التنظيف: {len(df)}")

📄 عدد المتطلبات بعد التنظيف: 20


In [ ]:
print(df.columns)

Index(['requirement', 'label'], dtype='object')


In [ ]:
pd.set_option('display.max_rows', None)
display(df)

,requirement,label
0,The owner can change the access code anytime.,Functional
1,The door should respond quickly.,Non-Functional
2,Only the tenant can unlock the door.,Functional
3,The system must support logging activities.,Functional
4,The app should look good.,Non-Functional
5,The door must auto-lock after 10 seconds.,Functional
6,Battery level should be visible to the owner.,Functional
7,The system must be available 24/7.,Non-Functional
8,The door should not unlock for unknown users.,Functional
9,The system should be reliable.,Non-Functional


In [ ]:
!pip install transformers datasets scikit-learn pandas torch

In [ ]:
import pandas as pd

# قراءة الملف
df = pd.read_csv("requirements.csv", encoding="utf-8")

# تنظيف البيانات من الأسطر الفارغة
df = df.dropna(subset=["requirement"])

# تحويل التصنيفات من نص إلى أرقام
label_map = {"Functional": 0, "Non-Functional": 1}
df["label"] = df["label"].map(label_map)

# عرض البيانات بعد التحويل
print(df.head())
print(f"\n✅ عدد المتطلبات: {len(df)}")

                                     requirement  label
0  The owner can change the access code anytime.      0
1               The door should respond quickly.      1
2           Only the tenant can unlock the door.      0
3    The system must support logging activities.      0
4                      The app should look good.      1

✅ عدد المتطلبات: 20


In [ ]:
from datasets import Dataset

dataset = Dataset.from_pandas(df[["requirement", "label"]])
dataset = dataset.train_test_split(test_size=0.2)

print(dataset)

DatasetDict({
    train: Dataset({
        features: ['requirement', 'label'],
        num_rows: 16
    })
    test: Dataset({
        features: ['requirement', 'label'],
        num_rows: 4
    })
})


In [ ]:
from transformers import AutoTokenizer

model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(batch["requirement"], truncation=True, padding=True)

tokenized_dataset = dataset.map(tokenize, batched=True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/16 [00:00<?, ? examples/s]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=4,
    weight_decay=0.01,
    logging_steps=10,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
)

In [ ]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss


TrainOutput(global_step=8, training_loss=0.582212507724762, metrics={'train_runtime': 33.5736, 'train_samples_per_second': 1.906, 'train_steps_per_second': 0.238, 'total_flos': 361777701120.0, 'train_loss': 0.582212507724762, 'epoch': 4.0})

In [ ]:
texts = [
    "The door should lock automatically after 10 seconds.",
    "The system must be reliable.",
    "The user can add multiple access codes."
]

inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)
outputs = model(**inputs)
preds = outputs.logits.argmax(dim=1).tolist()

reverse_map = {0: "Functional", 1: "Non-Functional"}
for text, pred in zip(texts, preds):
    print(f"{reverse_map[pred]:<17} → {text}")

Functional        → The door should lock automatically after 10 seconds.
Non-Functional    → The system must be reliable.
Functional        → The user can add multiple access codes.


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# نحصل على التوقعات للنموذج على بيانات الاختبار
predictions = trainer.predict(tokenized_dataset["test"])

# نحول القيم من مصفوفات إلى أرقام عادية
pred_labels = np.argmax(predictions.predictions, axis=1)
true_labels = predictions.label_ids

# حساب المقاييس
accuracy = accuracy_score(true_labels, pred_labels)
precision = precision_score(true_labels, pred_labels)
recall = recall_score(true_labels, pred_labels)
f1 = f1_score(true_labels, pred_labels)

# عرض النتائج
print("📊 Model Evaluation Metrics:")
print(f"✅ Accuracy:  {accuracy:.2f}")
print(f"✅ Precision: {precision:.2f}")
print(f"✅ Recall:    {recall:.2f}")
print(f"✅ F1 Score:  {f1:.2f}")

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


📊 Model Evaluation Metrics:
✅ Accuracy:  0.50
✅ Precision: 0.00
✅ Recall:    0.00
✅ F1 Score:  0.00


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# نحفظ النموذج والمحول (Tokenizer)
model.save_pretrained("/content/SmartLock_Model")
tokenizer.save_pretrained("/content/SmartLock_Model")

print("✅ تم حفظ النموذج والمحول بنجاح في مجلد SmartLock_Model")

✅ تم حفظ النموذج والمحول بنجاح في مجلد SmartLock_Model


In [ ]:
!zip -r SmartLock_Model.zip SmartLock_Model

  adding: SmartLock_Model/ (stored 0%)
  adding: SmartLock_Model/config.json (deflated 49%)
  adding: SmartLock_Model/special_tokens_map.json (deflated 42%)
  adding: SmartLock_Model/vocab.txt (deflated 53%)
  adding: SmartLock_Model/model.safetensors (deflated 7%)
  adding: SmartLock_Model/tokenizer.json (deflated 71%)
  adding: SmartLock_Model/tokenizer_config.json (deflated 75%)


In [ ]:
from google.colab import files
files.download("SmartLock_Model.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!ls -lh

total 387M
-rw-r--r-- 1 root root 1011 Oct  7 19:36 requirements.csv
drwxr-xr-x 3 root root 4.0K Oct  7 20:06 results
drwxr-xr-x 1 root root 4.0K Oct  6 13:38 sample_data
drwxr-xr-x 2 root root 4.0K Oct  7 20:12 SmartLock_Model
-rw-r--r-- 1 root root 387M Oct  7 20:13 SmartLock_Model.zip


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
)
trainer.train()

In [24]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch

texts = df["requirement"].tolist()
labels = df["label"].tolist()

# تحويل النصوص إلى مدخلات للنموذج
inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")

# إجراء التنبؤ
with torch.no_grad():
    outputs = model(**inputs)
preds = outputs.logits.argmax(dim=1).numpy()

# حساب المقاييس
precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
acc = accuracy_score(labels, preds)

print(f"📊 دقة النموذج: {acc:.2f}")
print(f"🎯 Precision: {precision:.2f}")
print(f"🔁 Recall: {recall:.2f}")
print(f"🏆 F1-score: {f1:.2f}")

📊 دقة النموذج: 0.90
🎯 Precision: 0.92
🔁 Recall: 0.90
🏆 F1-score: 0.90


In [25]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
